In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
dir_ELUC_NGHGI = '/GCB2021_commentary/Data/data_ELUC_NGHGI/'
dir_ctrs       = '/Trendy/Data/data_ancillary/info_countries/'
dir_ELUC_2021  = '/GCB2021_commentary/Data/ELUC_countries/'
dir_peat       = '/GCB2021/Data/Peat_data/'


## Define time and read country data

In [ ]:
#Define time
time_sta = 1750
time_end = 2020

#Read country names
fname_cntrs = dir_ctrs + 'Country codes 3 letters.xlsx'
data_cntrs  = pd.read_excel(fname_cntrs, sheet_name=0, header=None, index_col=0)

#Read all OSCAR regions and the missing OSCAR regions
OSCAR_countries = pd.read_csv(dir_ELUC_NGHGI + 'OSCAR_regions.csv')
OSCAR_countries = OSCAR_countries[['Country', 'ISO-Alpha3', 'Finest']]
OSCAR_regions   = pd.read_excel(dir_ELUC_NGHGI + 'OSCAR_all_regions.xlsx', header=0, index_col=0)


## Read data

In [ ]:
#Read source and sink data for BLUE, H&N2021, and OSCAR
fname_BLUE_snk  = dir_ELUC_2021 + 'ELUC_BLUE_countries-ISOcode_ELUC-sinks_vRemapCountries_1750-2020.xlsx'
fname_HN21_snk  = dir_ELUC_NGHGI + 'HN2021_ELUC-sinks_GCB2021_countries.xlsx'
fname_OSCAR_snk = dir_ELUC_NGHGI + 'OSCAR_ELUC-sinks_GCB2021_countries.xlsx'
fname_BLUE_src  = dir_ELUC_2021 + 'ELUC_BLUE_countries-ISOcode_ELUC-sources_vRemapCountries_1750-2020.xlsx'
fname_HN21_src  = dir_ELUC_NGHGI + 'HN2021_ELUC-sources_GCB2021_countries.xlsx'
fname_OSCAR_src = dir_ELUC_NGHGI + 'OSCAR_ELUC-sources_GCB2021_countries.xlsx'
data_BLUE_snk  = pd.read_excel(fname_BLUE_snk, header=0, index_col=0)
data_HN21_snk  = pd.read_excel(fname_HN21_snk, header=0, index_col=0)
data_OSCAR_snk = pd.read_excel(fname_OSCAR_snk, header=0, index_col=0)
data_BLUE_src  = pd.read_excel(fname_BLUE_src, header=0, index_col=0)
data_HN21_src  = pd.read_excel(fname_HN21_src, header=0, index_col=0)
data_OSCAR_src = pd.read_excel(fname_OSCAR_src, header=0, index_col=0)

#Select time
data_BLUE_snk  = data_BLUE_snk.loc[(data_BLUE_snk.index>=time_sta) & (data_BLUE_snk.index<=time_end)]
data_HN21_snk  = data_HN21_snk.loc[(data_HN21_snk.index>=time_sta) & (data_HN21_snk.index<=time_end)]
data_OSCAR_snk = data_OSCAR_snk.loc[(data_OSCAR_snk.index>=time_sta) & (data_OSCAR_snk.index<=time_end)]
data_BLUE_src  = data_BLUE_src.loc[(data_BLUE_src.index>=time_sta) & (data_BLUE_src.index<=time_end)]
data_HN21_src  = data_HN21_src.loc[(data_HN21_src.index>=time_sta) & (data_HN21_src.index<=time_end)]
data_OSCAR_src = data_OSCAR_src.loc[(data_OSCAR_src.index>=time_sta) & (data_OSCAR_src.index<=time_end)]

#Adjust some country names for H&N2021
for HN21_ctr in data_HN21_src.columns:
    
    if       "Bolivia" in HN21_ctr:                                       HN21_rep = "Bolivia (Plurinational State of)"
    elif     "Democratic Republic of the Congo" in HN21_ctr:              HN21_rep = "Congo, Democratic Republic of the"
    elif     "Cote d'Ivoire" in HN21_ctr:                                 HN21_rep = "Côte d'Ivoire"
    elif     "Czech Republic" in HN21_ctr:                                HN21_rep = "Czechia"
    elif     "Guinea Bissau" in HN21_ctr:                                 HN21_rep = "Guinea-Bissau"
    elif     "Iran" in HN21_ctr:                                          HN21_rep = "Iran (Islamic Republic of)"
    elif     "Democratic People's Republic of the Korea" in HN21_ctr:     HN21_rep = "Korea (Democratic People's Republic of)"
    elif     "Republic of Korea" in HN21_ctr:                             HN21_rep = "Korea, Republic of"
    elif     "The former Yugoslav Republic of Macedonia" in HN21_ctr:     HN21_rep = "North Macedonia"
    elif     "Republic of Moldova" in HN21_ctr:                           HN21_rep = "Moldova, Republic of"
    elif     "United Republic of Tanzania" in HN21_ctr:                   HN21_rep = "Tanzania, United Republic of"
    elif     "United Kingdom" in HN21_ctr:                                HN21_rep = "United Kingdom of Great Britain and Northern Ireland"
    elif     "Venezuela" in HN21_ctr:                                     HN21_rep = "Venezuela (Bolivarian Republic of)"
    elif     "Cape Verde" in HN21_ctr:                                    HN21_rep = "Cabo Verde"
    else:                                                                 continue
        
    #Rename countries
    data_HN21_src = data_HN21_src.rename(columns={HN21_ctr: HN21_rep})
    data_HN21_snk = data_HN21_snk.rename(columns={HN21_ctr: HN21_rep})
    

## Read peat data

In [ ]:
#Read peat data
fname_peat  = dir_peat + 'Country_ELUC_with_peat.xlsx'
data_peat_in_1 = pd.read_excel(fname_peat, sheet_name='FAO_Peat_Drainage', header=0, index_col=0)
data_peat_in_2 = pd.read_excel(fname_peat, sheet_name='GFED_Peat_Burning', header=0, index_col=0)
data_peat_in_2 = data_peat_in_2[[col for col in data_peat_in_2.columns if 'Unnamed' not in col]]
data_peat_in_1 = data_peat_in_1.loc[(data_peat_in_1.index>=time_sta) & (data_peat_in_1.index<=time_end)]
data_peat_in_2 = data_peat_in_2.loc[(data_peat_in_2.index>=time_sta) & (data_peat_in_2.index<=time_end)]


## Calculation

In [ ]:
#Get IPCC countries
countries = data_BLUE_snk.columns

#Create empty dataframes for storing scaled OSCAR data
data_ELUC_OSCAR_2021_snk = pd.DataFrame(columns=countries)
data_ELUC_OSCAR_2021_src = pd.DataFrame(columns=countries)
data_flag = pd.DataFrame(0, index=countries, columns=['BLUE', 'H&N2021', 'OSCAR'])

#Lists for checking H&N2021 countries
HN21_countries_missing = []
HN21_countries_skipped = set(data_HN21_snk.columns)

#Loop over countires
for country in countries:
    
    country_info = OSCAR_countries[OSCAR_countries['ISO-Alpha3']==country]
    
    #Skip country if not in OSCAR list
    if len(country_info)==0:
        print(country)
        continue
    
    #Get number and names of region and countries
    OSCAR_num  = country_info['Finest'].values[0]
    OSCAR_ctrs = OSCAR_countries[OSCAR_countries['Finest']==OSCAR_num]
    OSCAR_reg  = OSCAR_regions[OSCAR_regions['number']==OSCAR_num]
    HN21_ctrs  = data_cntrs.loc[OSCAR_ctrs['ISO-Alpha3']]
    
    #Add flag entry for OSCAR
    if len(OSCAR_ctrs)==1:
        data_flag.loc[country, 'OSCAR'] = 1
    
    #Exclude Antarctica (because it would lead to an error with BLUE)
    OSCAR_ctrs = OSCAR_ctrs[OSCAR_ctrs['ISO-Alpha3']!='ATA']
    
    #Read OSCAR region data
    OSCAR_snk_sel = data_OSCAR_snk[OSCAR_reg.name]
    OSCAR_src_sel = data_OSCAR_src[OSCAR_reg.name]

    #Select countries for H&N2021
    HN21_ctrs_sel = []
    for index, HN21_ctr in HN21_ctrs.iterrows():
        if HN21_ctr.values[0] in data_HN21_src.columns:
            HN21_ctrs_sel.append(HN21_ctr.values[0])
            HN21_countries_skipped.discard(HN21_ctr.values[0])
        else:
            HN21_countries_missing.append(HN21_ctr.values[0])
         
    #Select H&N2021 country data
    HN21_snk_sel = data_HN21_snk[HN21_ctrs_sel]
    HN21_src_sel = data_HN21_src[HN21_ctrs_sel] 
    
    #Select BLUE country data
    BLUE_snk_sel = data_BLUE_snk[OSCAR_ctrs['ISO-Alpha3']]
    BLUE_src_sel = data_BLUE_src[OSCAR_ctrs['ISO-Alpha3']]    
    
    #Calculate sum over reference datasets
    ctr_HN21 = data_cntrs.loc[country].values[0]
    if ctr_HN21 in HN21_snk_sel.columns:
        SUM_snk = (BLUE_snk_sel[country] + HN21_snk_sel[ctr_HN21]) / 2
        SUM_src = (BLUE_src_sel[country] + HN21_src_sel[ctr_HN21]) / 2
        TOT_snk = (BLUE_snk_sel.sum(axis=1) + HN21_snk_sel.sum(axis=1))/2
        TOT_src = (BLUE_src_sel.sum(axis=1) + HN21_src_sel.sum(axis=1))/2
        data_flag.loc[country, 'BLUE'] = 1
        data_flag.loc[country, 'H&N2021'] = 1
    else:
        SUM_snk = BLUE_snk_sel[country]
        SUM_src = BLUE_src_sel[country]
        TOT_snk = BLUE_snk_sel.sum(axis=1)
        TOT_src = BLUE_src_sel.sum(axis=1)
        data_flag.loc[country, 'BLUE'] = 1
        
    #Calculate scale factor
    scal_fac_snk = SUM_snk / TOT_snk
    scal_fac_src = SUM_src / TOT_src
    
    #Apply scale factor
    ELUC_scaled_OSCAR_snk = scal_fac_snk * OSCAR_snk_sel.iloc[:,0]
    ELUC_scaled_OSCAR_src = scal_fac_src * OSCAR_src_sel.iloc[:,0]

    #Add peat if necessary
    if country in data_peat_in_1.columns:
        ELUC_scaled_OSCAR_src = ELUC_scaled_OSCAR_src.add(data_peat_in_1[country], fill_value=0)
    if country in data_peat_in_2.columns:
        ELUC_scaled_OSCAR_src = ELUC_scaled_OSCAR_src.add(data_peat_in_2[country], fill_value=0)

    #Save in OSCAR data frame
    data_ELUC_OSCAR_2021_snk[country] = ELUC_scaled_OSCAR_snk
    data_ELUC_OSCAR_2021_src[country] = ELUC_scaled_OSCAR_src

#Calculate net flux
data_ELUC_OSCAR_2021_net = data_ELUC_OSCAR_2021_snk + data_ELUC_OSCAR_2021_src

#Add units
data_ELUC_OSCAR_2021_snk = data_ELUC_OSCAR_2021_snk.rename_axis('unit: Tg C/year')
data_ELUC_OSCAR_2021_src = data_ELUC_OSCAR_2021_src.rename_axis('unit: Tg C/year')
data_ELUC_OSCAR_2021_net = data_ELUC_OSCAR_2021_net.rename_axis('unit: Tg C/year')

#Save in files
fname_snk = dir_ELUC_NGHGI + 'OSCAR_ELUC-sinks_GCB2021_IPCCcountries.xlsx'
fname_src = dir_ELUC_NGHGI + 'OSCAR_ELUC-sources-with-peat_GCB2021_IPCCcountries.xlsx'
fname_net = dir_ELUC_NGHGI + 'OSCAR_ELUC-net-with-peat_GCB2021_IPCCcountries.xlsx'
data_ELUC_OSCAR_2021_snk.to_excel(fname_snk)
data_ELUC_OSCAR_2021_src.to_excel(fname_src)
data_ELUC_OSCAR_2021_net.to_excel(fname_net)

#Save flag in file
fname_flag = dir_ELUC_NGHGI + 'OSCAR_flag_scaling_IPCCcountries.xlsx'
data_flag.to_excel(fname_flag)
